In [83]:
import configparser

config = configparser.ConfigParser()
config.read("config.cfg")
BETA_PATH = config["DATA"]["BetaPath"]
output_dir = "/home/liancheng/CSD_manifold/output"
INFO_PATH = config["DATA"]["StimuliInfo"]
FREE_PATH = config["DATA"]["FreesurferPath"]

## 存储每个被试的合并数据

In [13]:
import h5py
from tqdm import tqdm
import numpy as np
import os

for sub in range(1,9):
    print('subject',sub)
    lh_beta_mat = None
    rh_beta_mat = None

    for run in tqdm(range(1,201)): # 200

        lh_file_path = "%s/sub-%02d/sub-%02d_task-CSD_run-%03d_lh.hdf5" % (BETA_PATH, sub, sub, run)
        rh_file_path = "%s/sub-%02d/sub-%02d_task-CSD_run-%03d_rh.hdf5" % (BETA_PATH, sub, sub, run)
        # 打开 hdf5 文件

        try:
            with h5py.File(lh_file_path, 'r') as f:
                # 打印文件中的所有键（类似文件夹结构）
                lh_beta = f['betas'][:]
                if lh_beta_mat is None:
                    lh_beta_mat = lh_beta
                else:
                    lh_beta_mat = np.vstack((lh_beta_mat, lh_beta))

            with h5py.File(rh_file_path, 'r') as f:
                # 打印文件中的所有键（类似文件夹结构）
                rh_beta = f['betas'][:]
                if rh_beta_mat is None:
                    rh_beta_mat = rh_beta
                else:
                    rh_beta_mat = np.vstack((rh_beta_mat, rh_beta))

        except FileNotFoundError:
            break

    if not os.path.isdir("%s/sub-%02d" % (output_dir, sub)):
            os.makedirs("%s/sub-%02d" % (output_dir, sub))


    np.save(
                "%s/sub-%02d/rh_beta_sub-%02d.npy"
                % (output_dir, sub, sub),
                rh_beta_mat,
            )
    np.save(
                "%s/sub-%02d/lh_beta_sub-%02d.npy"
                % (output_dir, sub, sub),
                lh_beta_mat,
            )


subject 1


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [05:00<00:00,  1.50s/it]


subject 2


100%|██████████| 200/200 [05:10<00:00,  1.55s/it]


...

100%|██████████| 200/200 [05:31<00:00,  1.66s/it]


subject 7


100%|██████████| 200/200 [05:16<00:00,  1.58s/it]


subject 8


100%|██████████| 200/200 [04:49<00:00,  1.45s/it]


In [27]:
import pandas as pd
for sub in range(1,9):
    merged = None
    for run in tqdm(range(1,201)):
        info_path = "%s/sub-%02d/subject%d_run%d.txt" % (INFO_PATH, sub, sub, run)
        info = pd.read_csv(info_path, encoding='GBK', sep='\t')
        info_clean = info[info['Blank'] != 1]   
        info_clean = info_clean[['Unmatch', 'Image', 'Caption']]
        if merged is None:
            merged = info_clean
        else:
            merged = pd.concat([merged,info_clean],axis=0).reset_index(drop=True)
    merged.to_csv("%s/sub-%02d/stimuli_info_sub-%02d.txt"
                % (output_dir, sub, sub),
                  sep='\t', encoding='utf-8')


100%|██████████| 200/200 [00:00<00:00, 588.73it/s]


## 处理coco

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
indoor vehicle animal kitchen furniture outdoor accessory person food electronic appliance sports

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
from collections import Counter

coco_train = COCO('annotations/instances_train2014.json')
coco_val = COCO('annotations/instances_val2014.json')

loading annotations into memory...
Done (t=7.06s)
creating index...
index created!
loading annotations into memory...
Done (t=3.81s)
creating index...
index created!


('sports', 'sports ball')

In [68]:
from collections import Counter
def get_center_supcat(coco_train,coco_val,file_name):
    """ 
    离注视点最近的超类别和类别 
    
    输入：
    coco_train coco_val：即load好的实例类
    file_name：来自stimuli info的图片文件名

    输出：离注视点最近的超类别和类别
    """

    img_id = int(file_name.split('_')[-1].split('.')[0])
    if 'train' in file_name:
        coco = coco_train
    elif 'val' in file_name :
        coco = coco_val
    else:
        raise NameError
    
    
    img_info = coco.loadImgs(img_id)[0]
    center_x, center_y = img_info['width'] / 2, img_info['height'] / 2


    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    if not anns:
        return None 


    min_dist = float('inf')
    closest_ann = None

    for ann in anns:
        x, y, w, h = ann['bbox']  # COCO bbox = [x, y, width, height]
        ann_center_x = x + w / 2
        ann_center_y = y + h / 2
        dist = np.sqrt((ann_center_x - center_x)**2 + (ann_center_y - center_y)**2)
        if dist < min_dist:
            min_dist = dist
            closest_ann = ann

    
    if closest_ann:
        cat_info = coco.loadCats(closest_ann['category_id'])[0]
        return cat_info['supercategory']
    else:
        return None
    
def get_frequent_supcat(coco_train,coco_val,file_name):
    """ 
    图片中出现最多的超类别和类别 
    
    输入：
    coco_train coco_val：即load好的实例类
    file_name：来自stimuli info的图片文件名

    输出：图片中出现最多的超类别和类别 
    """

    img_id = int(file_name.split('_')[-1].split('.')[0])
    if 'train' in file_name:
        coco = coco_train
    elif 'val' in file_name :
        coco = coco_val
    else:
        raise NameError
    

    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    if not anns:
        return None

    category_ids = [ann['category_id'] for ann in anns]
    cat_counter = Counter(category_ids)
    
    most_common_id, count = cat_counter.most_common(1)[0]
    if most_common_id:
        cat_info = coco.loadCats(most_common_id)[0] 
        return cat_info['supercategory']
    else:
        return None

In [69]:
sub = 1 
df = pd.read_csv("%s/sub-%02d/stimuli_info_sub-%02d.txt"
                % (output_dir, sub, sub),
                  sep='\t', encoding='utf-8',index_col = 0)
df['center_cat'] = df['Image'].apply(lambda x: get_center_supcat(coco_train, coco_val, x))
df['frequent_cat'] = df['Image'].apply(lambda x: get_frequent_supcat(coco_train, coco_val, x))

In [ ]:
df

,Unmatch,Image,Caption,center_cat,frequent_cat
0,0,COCO_train2014_000000219229.jpg,围栏里一群羊懒洋洋的躺着。,animal,animal
1,0,COCO_train2014_000000389352.jpg,一个平坦的滑雪场内分布着许多人。,sports,person
2,0,COCO_val2014_000000346774.jpg,餐桌上的一位女士面前放着一个大比萨饼。,kitchen,furniture
3,0,COCO_train2014_000000016669.jpg,桌子上放着一台电脑和一个台灯,electronic,electronic
4,0,COCO_train2014_000000435414.jpg,一个男人和两只狗在公园中散步,animal,animal
...,...,...,...,...,...
4395,0,COCO_train2014_000000166207.jpg,一男一女在街道上并肩行走,person,person
4396,0,COCO_train2014_000000230289.jpg,一个人站在山顶正准备向下滑雪,person,person
4397,0,COCO_train2014_000000298835.jpg,几个游客在长椅上休息。,person,person
4398,0,COCO_val2014_000000343218.jpg,一个男人在网球场里挥起球拍打球,sports,sports


In [99]:
sub = 1
df.to_csv("%s/sub-%02d/stimuli_info_sub-%02d_cat.txt"
                % (output_dir, sub, sub),
                  sep='\t', encoding='utf-8')

In [79]:
person_indices = df[(df['center_cat'] == 'person') & 
                    (df['Unmatch'] == 0)].index.tolist()
len(person_indices)


1024

In [80]:
import nibabel.freesurfer.io as fsio
import nibabel as nib
import numpy as np

vcAtlas = ['hOc1','hOc2','hOc3v','hOc4v','FG1','FG2','FG3','FG4']

In [94]:
sub = 1
lh_beta = np.load("%s/sub-%02d/lh_beta_sub-%02d.npy"
                % (output_dir, sub, sub))
rh_beta = np.load("%s/sub-%02d/rh_beta_sub-%02d.npy"
                % (output_dir, sub, sub))
for i in range(8):
    atla = vcAtlas[i]

    lh_label_path = "%s/sub-%02d/label/lh.%s.mpm.vpnl.label" % (FREE_PATH, sub, atla)
    rh_label_path = "%s/sub-%02d/label/rh.%s.mpm.vpnl.label" % (FREE_PATH, sub, atla)
    lh_label= fsio.read_label(lh_label_path)  # verts 是顶点编号数组
    rh_label= fsio.read_label(rh_label_path) 

    np.save(
                "%s/sub-%02d/lh_beta_sub-%02d_%s.npy"
                % (output_dir, sub, sub, atla),
                lh_beta[:,lh_label],
            )
    np.save(
                "%s/sub-%02d/rh_beta_sub-%02d_%s.npy"
                % (output_dir, sub, sub, atla),
                rh_beta[:,rh_label],
            )



In [ ]:
from manifold.mftma.manifold_analysis_correlation import manifold_analysis_corr

for layer, data, in activations.items():
    X = [d.reshape(d.shape[0], -1).T for d in data]
    # Get the number of features in the flattened data
    N = X[0].shape[0]
    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M*M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X

capacities = []
radii = []
dimensions = []
correlations = []

for k, X, in activations.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    
    # Compute the mean values
    a = 1/np.mean(1/a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))
    
    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)


ModuleNotFoundError: No module named 'pymanopt.solvers'